In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import requests
import sys

import tensorflow as tf

tf.random.set_seed(1234)
AUTO = tf.data.experimental.AUTOTUNE

#import tensorflow_datasets as tfds

import os
import re
import numpy as np
from time import time
import matplotlib.pyplot as plt

print("Tensorflow version {}".format(tf.__version__))
print("Numpy version {}".format(np.__version__))

Tensorflow version 2.5.0
Numpy version 1.17.0


In [24]:
#model requires python 3.6 
!python --version

Python 3.6.13 :: Anaconda, Inc.


In [3]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [4]:
import pandas as pd

df_train, df_test = pd.read_csv('atis_intents_train.csv'), pd.read_csv('atis_intents_test.csv')
df_train.columns, df_test.columns = ['intent', 'snippet'], ['intent', 'snippet']

df_train.head()

,intent,snippet
0,atis_flight,what flights are available from pittsburgh to...
1,atis_flight_time,what is the arrival time in san francisco for...
2,atis_airfare,cheapest airfare from tacoma to orlando
3,atis_airfare,round trip fares from pittsburgh to philadelp...
4,atis_flight,i need a flight tomorrow from columbus to min...


In [5]:
df_train.intent.value_counts(), df_train.intent.value_counts(normalize=True)

(atis_flight            3665
 atis_airfare            423
 atis_ground_service     255
 atis_airline            157
 atis_abbreviation       147
 atis_aircraft            81
 atis_flight_time         54
 atis_quantity            51
 Name: intent, dtype: int64,
 atis_flight            0.758328
 atis_airfare           0.087523
 atis_ground_service    0.052762
 atis_airline           0.032485
 atis_abbreviation      0.030416
 atis_aircraft          0.016760
 atis_flight_time       0.011173
 atis_quantity          0.010552
 Name: intent, dtype: float64)

In [6]:
train_data = df_train.snippet.values
train_labels = df_train.intent.values
test_data = df_test.snippet.values
test_labels = df_test.intent.values          

print('Training Dataset Size: {}'.format(len(train_data)))
#print('Train Targets: {}'.format(len(train_labels)))
print('Testing Dataset Size: {}'.format(len(test_data)))
#print('Test Targets: {}'.format(len(test_labels)))
print('Dataset Type: {}'.format(type(train_data)))

Training Dataset Size: 4833
Testing Dataset Size: 799
Dataset Type: <class 'numpy.ndarray'>


In [7]:
# Maximum sentence length
MAX_LENGTH = 10

tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(list(df_train.snippet) + list(df_test.snippet))
VOCAB_SIZE = tokenizer.document_count
tokenized_requests_train = tokenizer.texts_to_sequences(df_train.snippet)
tokenized_requests_train = tf.keras.preprocessing.sequence.pad_sequences(tokenized_requests_train, 
                                                                    maxlen=MAX_LENGTH,padding='post')
tokenized_requests_test = tokenizer.texts_to_sequences(df_test.snippet)
tokenized_requests_test = tf.keras.preprocessing.sequence.pad_sequences(tokenized_requests_test, 
                                                                    maxlen=MAX_LENGTH,padding='post')

In [8]:
print('Sample Request: {}'.format(train_data[2]))
print('Sample Token: {}'.format(tokenized_requests_train[2]))
print('Sample Target: {}'.format(train_labels[2]))

Sample Request:  cheapest airfare from tacoma to orlando
Sample Token: [ 53 260   2 164   1  95   0   0   0   0]
Sample Target: atis_airfare


In [9]:
target_train_df = pd.get_dummies(train_labels)
target_test_df = pd.get_dummies(test_labels)
target_train_df.head()

,atis_abbreviation,atis_aircraft,atis_airfare,atis_airline,atis_flight,atis_flight_time,atis_ground_service,atis_quantity
0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,0


In [10]:
tokenized_targets_train = target_train_df.values
tokenized_targets_test = target_test_df.values
print('Dataset Type: {}'.format(type(tokenized_targets_train)))

Dataset Type: <class 'numpy.ndarray'>


In [11]:
# Train/Val Split
prop = 0.8
N = len(tokenized_requests_train)
x_train = tokenized_requests_train[:int(N*prop)]
y_train = tokenized_targets_train[:int(N*prop)]

x_val = tokenized_requests_train[int(N*prop):]
y_val = tokenized_targets_train[int(N*prop):]

x_test = tokenized_requests_test
y_test = tokenized_targets_test

In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(VOCAB_SIZE,100))
model.add(tf.keras.layers.Bidirectional((tf.keras.layers.LSTM(64, return_sequences=True))))
model.add(tf.keras.layers.Bidirectional((tf.keras.layers.LSTM(64))))
model.add(tf.keras.layers.Dense(8, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         563200    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         84480     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 8)                 1032      
Total params: 747,528
Trainable params: 747,528
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [14]:
history = model.fit(
    x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val)
)

Epoch 1/5
121/121 [==============================] - 10s 37ms/step - loss: 0.2314 - accuracy: 0.7618 - val_loss: 0.1278 - val_accuracy: 0.8097
Epoch 2/5
121/121 [==============================] - 3s 26ms/step - loss: 0.1133 - accuracy: 0.8409 - val_loss: 0.0979 - val_accuracy: 0.8645
Epoch 3/5
121/121 [==============================] - 3s 23ms/step - loss: 0.0888 - accuracy: 0.8808 - val_loss: 0.0796 - val_accuracy: 0.9007
Epoch 4/5
121/121 [==============================] - 3s 27ms/step - loss: 0.0703 - accuracy: 0.9077 - val_loss: 0.0770 - val_accuracy: 0.9080
Epoch 5/5
121/121 [==============================] - 3s 23ms/step - loss: 0.0589 - accuracy: 0.9265 - val_loss: 0.0674 - val_accuracy: 0.9245


In [ ]:
# VALIDATION LOSS curves

plt.clf()
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, (len(history_dict['loss']) + 1))
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:

# VALIDATION ACCURACY curves

plt.clf()
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, (len(history_dict['accuracy']) + 1))
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [16]:
print(x_test.shape,y_test.shape)

(799, 10) (799, 8)


In [17]:
print('Sample Request: {}'.format(test_data[2]))
print('Sample Request Token: {}'.format(x_test[2]))
print('Sample Target: {}'.format(test_labels[2]))
print('Sample Target Token: {}'.format(y_test[2]))

Sample Request:  i would like a flight traveling one way from phoenix to san diego on april first
Sample Request Token: [ 69  72   2  92   1  11 101   5 186  43]
Sample Target: atis_flight
Sample Target Token: [0 0 0 0 1 0 0 0]


In [18]:
y_pred = model.predict(x_test)

In [19]:
y_pred.shape

(799, 8)

In [21]:
y_pred[2,:]

array([8.3178875e-06, 2.2383257e-04, 4.3115282e-04, 3.2387496e-04,
       9.9874699e-01, 1.4744427e-04, 3.1680276e-05, 8.6684879e-05],
      dtype=float32)